### 1

In [1]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
user_question = "I just discovered the course. Can I still join it?"
vector = model.encode(user_question)

In [51]:
vector.shape

(768,)

In [5]:
vector[0]

0.07822261

### 2

In [6]:
import requests

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [10]:
hw_documents = [doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"]

In [11]:
len(hw_documents)

375

In [12]:
from tqdm import tqdm

embeddings = []

for doc in tqdm(hw_documents):
    question = doc["question"]
    text = doc["text"]
    qa_text = f"{question} {text}"

    doc["question_vector"] = model.encode(question)
    doc["text_vector"] = model.encode(text)
    doc["question_text_vector"] = model.encode(qa_text)

    embeddings.append(doc["question_text_vector"])

100%|██████████| 375/375 [01:18<00:00,  4.79it/s]


In [13]:
import numpy as np

X = np.array(embeddings)

In [14]:
X.shape

(375, 768)

### 3

In [17]:
scores = X.dot(vector)

In [19]:
scores.max()

0.6506574

In [20]:
idx = np.argsort(-scores)[:1]
idx

array([14])

In [21]:
scores[14]

0.6506574

In [94]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, query, num_results=5):
        v_query = model.encode(query)
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


search_engine = VectorSearchEngine(documents=hw_documents, embeddings=X)
result = search_engine.search(user_question, num_results=5)

### 4

In [25]:
import pandas as pd

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

In [35]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [43]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [101]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q["document"]
        results = search_function(q["question"])
        relevance = [d["id"] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        "hit_rate": hit_rate(relevance_total),
        "mrr": mrr(relevance_total),
    }

In [96]:
evaluate(ground_truth, search_engine.search)

100%|██████████| 1830/1830 [00:37<00:00, 49.21it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

### 5

In [52]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [53]:
for doc in tqdm(hw_documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:03<00:00, 104.45it/s]


In [97]:
def elastic_search_knn(field, vector, course="machine-learning-zoomcamp"):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {"term": {"course": course}},
    }

    search_query = {"knn": knn, "_source": ["text", "section", "question", "course", "id"]}

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [98]:
course = "machine-learning-zoomcamp"
result = elastic_search_knn("question_text_vector", vector, course)

In [99]:
result[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

### 6

In [100]:
def vector_combined_knn(q):
    v_q = model.encode(q)
    return elastic_search_knn("question_text_vector", v_q)


evaluate(ground_truth, vector_combined_knn)

100%|██████████| 1830/1830 [00:38<00:00, 47.35it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}